# Running Native Functions

Two of the previous notebooks showed how to [execute semantic functions inline](./03-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

* Writing logic around how to run a prompt that changes the prompt's outcome.
* Using external data sources to gather data to concatenate into your prompt.
* Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the SK Python framework.

Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [1]:
#!python -m pip install semantic-kernel==0.4.5.dev0

In [2]:
pip freeze

aiofiles==23.2.1
aiohttp==3.9.1
aiosignal==1.3.1
annotated-types==0.6.0
anyio==4.2.0
asgiref==3.7.2
asttokens==2.4.1
async-timeout==4.0.3
attrs==23.2.0
azure-common==1.1.28
azure-core==1.29.7
azure-identity==1.15.0
azure-search-documents==11.4.0
black==23.12.1
certifi==2023.11.17
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
click==8.1.7
comm==0.2.1
cryptography==41.0.7
dataclasses-json==0.6.3
debugpy==1.8.0
decorator==5.1.1
Deprecated==1.2.14
distro==1.9.0
dnspython==2.4.2
exceptiongroup==1.2.0
executing==2.0.1
frozenlist==1.4.1
fsspec==2023.12.2
greenlet==3.0.3
h11==0.14.0
httpcore==1.0.2
httpx==0.26.0
idna==3.6
importlib-metadata==7.0.1
ipykernel==6.29.0
ipython==8.18.1
isodate==0.6.1
jedi==0.19.1
joblib==1.3.2
jsonpatch==1.33
jsonpointer==2.4
jsonschema==4.21.0
jsonschema-path==0.3.2
jsonschema-spec==0.2.4
jsonschema-specifications==2023.7.1
jupyter_client==8.6.0
jupyter_core==5.7.1
langchain==0.1.1
langchain-community==0.0.13
langchain-core==0.1.13
langchain-openai==0.0.3


In [3]:
import os
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
embeddings = os.environ["AZURE_OPENAI_EMBEDDINGS_MODEL_NAME"]
azure_ai_search_api_key = os.environ["AZURE_AISEARCH_API_KEY"]
azure_ai_search_url = os.environ["AZURE_AISEARCH_ENDPOINT"]
#BING_API_KEY = os.environ["BING_API_KEY"]

In [4]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel

#deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
azure_chat_service = AzureChatCompletion(
    deployment_name=deployment, endpoint=endpoint, api_key=api_key
)  # set the deployment name to the value of your chat model
kernel.add_chat_service("chat_completion", azure_chat_service)


Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.

First, let's create our native function.

In [5]:
import random
#from semantic_kernel.plugin_definition import sk_function
from semantic_kernel.plugin_definition.sk_function_decorator import sk_function

class GenerateNumberPlugin:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Corgis on an adventure. `$input` is a default variable semantic functions can use. 

In [6]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long
"""

corgi_story = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    description="Write a short story about two Corgis on an adventure",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

generate_number_plugin = kernel.import_plugin(GenerateNumberPlugin())

In [7]:
# Run the number generator
generate_number_three_or_higher = generate_number_plugin["GenerateNumberThreeOrHigher"]
number_result = generate_number_three_or_higher(6)
print(number_result)

3


In [8]:
story = await corgi_story.invoke_async(input=number_result.result)

In [9]:
print("Generating a corgi story exactly {} paragraphs long: ".format(number_result.result))
print("=====================================================")
print(story)

Generating a corgi story exactly 3 paragraphs long: 
Once upon a time in a quaint little town, two adorable Corgis named Charlie and Daisy embarked on an exciting adventure. Their wagging tails and curious eyes were filled with anticipation as they set off to explore the nearby forest. As they trotted along the winding path, they encountered various creatures and breathtaking sights. Charlie and Daisy greeted each one with kindness and respect, making new friends along the way.

Their adventure took an unexpected turn when they stumbled upon a lost baby bird. It chirped helplessly, unable to find its way back home. Without hesitation, Charlie and Daisy worked together to build a cozy nest and gently placed the bird inside. They took turns keeping it warm and fed until its family returned. The grateful bird family sang a melodious tune to express their gratitude, filling the forest with joy.

As the sun began to set, Charlie and Daisy returned home, their hearts filled with contentment.

## Context Variables

That works! But let's expand on our example to make it more generic. 

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the `semantic_kernel.ContextVariables` class to do hold these variables.

In [10]:
# import semantic_kernel as sk
# from semantic_kernel.connectors.ai.open_ai import (
#     AzureChatCompletion,
#     OpenAIChatCompletion,
# )

# kernel = sk.Kernel()

# useAzureOpenAI = True


# # Configure AI service used by the kernel

# deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
# azure_chat_service = AzureChatCompletion(
#         deployment_name="turbo", endpoint=endpoint, api_key=api_key
#     )  # set the deployment name to the value of your chat model
# kernel.add_chat_service("chat_completion", azure_chat_service)


Let's start with the native function. Notice that we're also adding `@sk_function_context_parameter` decorators to the function here to provide context about what variables need to be provided to the function, and any defaults for those inputs. Using the `@sk_function_context_parameter` decorator provides the name, description and default values for a function's inputs to the [planner.](./05-using-the-planner.ipynb)

In [11]:
import random
from semantic_kernel.plugin_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext


class GenerateNumberPlugin:
    """
    Description: Generate a number between a min and a max.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber",
    )
    @sk_function_context_parameter(name="min", description="Minimum number of paragraphs.")
    @sk_function_context_parameter(name="max", description="Maximum number of paragraphs.", default_value="10")
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(int(context["min"]), int(context["max"])))
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

In [12]:
generate_number_plugin = kernel.import_plugin(GenerateNumberPlugin())
generate_number = generate_number_plugin["GenerateNumber"]

Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our CorgiStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.

In [13]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

corgi_story = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    description="Write a short story about two Corgis on an adventure",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

Now we can call this using our `invoke` function by passing in our `context_variables` in the `variables` parameter. 

In [14]:
context_variables = sk.ContextVariables(variables={"min": "1", "max": "5", "language": "Spanish"})

Let's add a paragraph count to our context variables 

In [15]:
context_variables["paragraph_count"] = generate_number.invoke(variables=context_variables).result

In [16]:
# Pass the output to the semantic story function
story = await corgi_story.invoke_async(variables=context_variables)

In [17]:
print(
    "Generating a corgi story exactly {} paragraphs long in {} language: ".format(
        context_variables["paragraph_count"], context_variables["language"]
    )
)
print("=====================================================")
print(story)

Generating a corgi story exactly 1 paragraphs long in Spanish language: 
Había una vez dos corgis llamados Max y Lola, quienes vivían en un pequeño pueblo rodeado de hermosos paisajes. Un día, decidieron embarcarse en una aventura para explorar el mundo más allá de su hogar. Juntos, caminaron por senderos desconocidos, saltaron sobre rocas y se maravillaron con la naturaleza que los rodeaba. Durante su viaje, conocieron a otros animales amigables y compartieron risas y juegos. A medida que avanzaban, Max y Lola se dieron cuenta de que la verdadera belleza de la vida radicaba en la amistad y la conexión con los demás. Al final de su aventura, regresaron a casa con corazones llenos de gratitud y una lección aprendida: la felicidad se encuentra en los momentos compartidos y en la apreciación de las pequeñas cosas. Desde entonces, Max y Lola siempre estuvieron dispuestos a embarcarse en nuevas aventuras, sabiendo que el mundo estaba lleno de maravillas por descubrir y amigos por hacer.


## Calling Native Functions within a Semantic Function

One neat thing about the Semantic Kernel is that you can also call native functions from within Semantic Functions!

We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}`. You can read more about our prompte templating syntax [here](../../../docs/PROMPT_TEMPLATE_LANGUAGE.md). 

In [18]:
import random
from semantic_kernel.plugin_definition import sk_function


class GenerateNamesPlugin:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @sk_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @sk_function(description="Generate character names", name="generate_names")
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {"Hoagie", "Hamilton", "Bacon", "Pizza", "Boots", "Shorts", "Tuna"}
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [19]:
generate_names_plugin = kernel.import_plugin(GenerateNamesPlugin(), plugin_name="GenerateNames")
generate_names = generate_names_plugin["generate_names"]

In [20]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The two names of the corgis are {{GenerateNames.generate_names}}
"""

In [21]:
corgi_story = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    description="Write a short story about two Corgis on an adventure",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

In [22]:
context_variables = sk.ContextVariables(variables={"min": "1", "max": "5", "language": "Spanish"})
context_variables["paragraph_count"] = generate_number.invoke(variables=context_variables).result

In [23]:
# Pass the output to the semantic story function
story = await corgi_story.invoke_async(variables=context_variables)

In [24]:
print(
    "Generating a corgi story exactly {} paragraphs long in {} language: ".format(
        context_variables["paragraph_count"], context_variables["language"]
    )
)
print("=====================================================")
print(story)

Generating a corgi story exactly 2 paragraphs long in Spanish language: 
Había una vez dos corgis llamados Hoagie y Bacon que vivían en un pequeño pueblo. Eran los mejores amigos y siempre estaban buscando aventuras juntos. Un día, decidieron explorar el bosque mágico que se encontraba al otro lado del río.

Mientras caminaban por el bosque, Hoagie y Bacon se encontraron con un conejito asustado. El conejito les contó que se había perdido y no podía encontrar su camino de regreso a casa. Los corgis, llenos de compasión, decidieron ayudar al conejito. Juntos, buscaron pistas y siguieron el rastro de las hojas caídas hasta que finalmente encontraron la madriguera del conejito. El conejito estaba tan agradecido que les prometió ser su amigo para siempre.

Hoagie y Bacon aprendieron que siempre es importante ayudar a los demás, sin importar cuán pequeños o grandes sean. Su amistad y espíritu de aventura les permitieron hacer nuevos amigos y descubrir la importancia de la bondad y la compas

### Recap

A quick review of what we've learned here:

- We've learned how to create native and semantic functions and register them to the kernel
- We've seen how we can use context variables to pass in more custom variables into our prompt
- We've seen how we can call native functions within semantic function prompts.  
